In [1]:
import open3d as o3d
import numpy as np
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('../sample')
from readdatas import *

In [3]:
path = '..\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [4]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [5]:
ground = pcd.ground
ground_o3d = gpf.np_to_o3d(pcd.ground)

In [6]:
class DriPathSegment2(BaseAlgorithm):

    def __init__(self,data):
        
        self.data = data
        self.ground = self.data.ground
        ground_o3d = self.np_to_o3d(ground)
        self.ground_nor = np.asarray(ground_o3d.normals)
        driving_track_seed = self.seed_select(ground_o3d)
        driving_track = self.driving_path_generation(driving_track_seed, ground_o3d)
        driving_track_o3d = self.np_to_o3d(driving_track)
        driving_track = self.find_nearest_point(ground_o3d, driving_track_o3d)
        self.density = self.calculate_linear_density(ground_o3d)
        driving_track_o3d = ground_o3d.select_by_index(driving_track)
        self.point_neighbours = self.find_nearest_point(driving_track_o3d,ground_o3d)
        self.driving_track = np.asarray(driving_track_o3d.points)       


    def _process_data(self):

        index_all = np.array(range(len(ground)))
        no_paves = np.array([0])
        paves = np.array([0])

        for i in range(len(self.driving_track)):
            slim = self.ground[self.point_neighbours[:,0] == i]
            slim_index = index_all[self.point_neighbours[:,0] == i]
            if len(slim) > 0:
                slim_nor = self.ground_nor[self.point_neighbours[:,0] == i]
                slim_distance = np.sqrt(np.sum(np.power(slim - self.driving_track[i,:],2),axis = 1))
                driving_track_point = slim_distance.argmin()
                driving_track_point = slim_nor[driving_track_point,:]
                included_angle = np.fabs(np.dot(slim_nor, driving_track_point))
                no_paves_slim = included_angle <= self.data.cosine_threshold
                paves_slim = included_angle > self.data.cosine_threshold

                slim_distance = slim_distance[paves_slim]
                if len(slim_distance) > 0:

                    paves_slim_h = slim[paves_slim,2].reshape(1,-1)
                    paves_slim = slim_index[paves_slim]
                    arr = np.argsort(paves_slim_h)
                    paves_slim_h = paves_slim_h[arr]
                    diff_arr = np.diff(paves_slim_h)
                    stop_index = np.where(diff_arr > 3 * self.density)[0][0] + 1
                    arr = arr[:stop_index]
                    paves_slim = paves_slim[arr]
                    slim_distance = slim_distance[arr]
                    arr = np.argsort(slim_distance)
                    slim_distance = np.sort(slim_distance)
                    slim_distance = slim_distance[arr]
                    diff_arr = np.diff(slim_distance)
                    stop_index = np.where(diff_arr > 5 * self.density)[0][0] + 1
                    paves_slim = paves_slim[:stop_index]
                    paves = np.concatenate((paves,paves_slim),axis=0)
                no_paves = np.concatenate((no_paves,slim_index[no_paves_slim]),axis=0)

        self.data.paves = np.delete(paves,0,0)     
        self.data.no_paves = np.delete(no_paves,0,0)

In [14]:
dps = DriPathSegment2(pcd)

[Open3D INFO] Picked point #15094 (-14., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #14564 (-6.5, -13., -1.6) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #31826 (-1.2, -12., -1.6) to add in queue.
[Open3D INFO] Picked point #56710 (2.9, -9.1, -1.5) to add in queue.
[Open3D INFO] Picked point #61957 (4.7, -5.3, -1.5) to add in queue.
[Open3D INFO] Picked point #69091 (5.1, -2.1, -1.5) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #68174 (5.3, 3.4, -1.5) to add in queue.
[Open3D INFO] Picked point #69518 (5.2, 5.9, -1.5) to add in queue.


In [15]:
def process_data(dps):

        index_all = np.array(range(len(ground)))
        no_paves = np.array([0])
        paves = np.array([0])

        for i in range(len(dps.driving_track)):
            slim = dps.ground[dps.point_neighbours[:,0] == i]
            slim_index = index_all[dps.point_neighbours[:,0] == i]
            if len(slim) > 0:
               slim_nor = dps.ground_nor[dps.point_neighbours[:,0] == i]
               slim_distance = np.sqrt(np.sum(np.power(slim - dps.driving_track[i,:],2),axis = 1))
               driving_track_point = slim_distance.argmin()
               driving_track_point = slim_nor[driving_track_point,:]
               included_angle = np.fabs(np.dot(slim_nor, driving_track_point))
               no_paves_slim = included_angle <= dps.data.cosine_threshold
               paves_slim = included_angle > dps.data.cosine_threshold

               slim_distance = slim_distance[paves_slim]
               if len(slim_distance) > 0:

                    paves_slim_h = slim[paves_slim,2].reshape(1,-1)
                    paves_slim = slim_index[paves_slim]
                    arr = np.argsort(paves_slim_h)[0]
                    paves_slim_h = np.sort(paves_slim_h)
                    diff_arr = np.diff(paves_slim_h)[0]
                    stop_index = np.where(diff_arr > 2 * dps.density)
                    if len(stop_index[0]) > 0:
                         stop_index = stop_index[0]
                         stop_index +=1
                         stop_index = stop_index[0]
                    else:
                         stop_index = paves_slim_h.shape[1]

                    
                    arr = arr[:stop_index]
                    paves_slim = paves_slim[arr]
                    slim_distance = slim_distance[arr]
                    arr = np.argsort(slim_distance)
                    slim_distance = np.sort(slim_distance)
                    diff_arr = np.diff(slim_distance)
                    stop_index = np.where(diff_arr > 2 * dps.density)
                    if len(stop_index[0]) > 0:
                         stop_index = stop_index[0]
                         stop_index +=1
                         stop_index = stop_index[0]
                    else:
                         stop_index = slim_distance.shape[0]
                    arr = arr[:stop_index]
                    paves_slim = paves_slim[arr]
                    paves = np.concatenate((paves, paves_slim),axis=0)
               # paves = np.concatenate((paves, slim_index[paves_slim]),axis=0)
               no_paves = np.concatenate((no_paves, slim_index[no_paves_slim]),axis=0)

        dps.data.paves = np.delete(paves,0,0)     
        dps.data.no_paves = np.delete(no_paves,0,0)

In [16]:
process_data(dps)

In [17]:
paves = ground_o3d.select_by_index(pcd.paves)
no_paves = ground_o3d.select_by_index(pcd.no_paves)
o3d.visualization.draw_geometries([paves])